In [ ]:
%pip install langchain llama-cpp-python 
#CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import os

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
# DEFAULT_SYSTEM_PROMPT = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information"
DEFAULT_SYSTEM_PROMPT = """
<<SYS>>
You are a chatbot that helps people learn a new language. You always give the answer to the question in a formal manner.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT +E_SYS

In [ ]:
SYSTEM_PROMPT_2 = """
<<SYS>>
You are a chatbot that always gives the answer to the question in a formal, short and very simple manner. You leave out unnecessary words and if the answer is only one word you don't put a dot and the end. If you are asked to translate something you translate it.
If you don't know the answer to a question, you tell them truthfully that you don't know and don't give false information. You are a helpful, respectful and honest assistant.
Your answers should not contain harmful, unethical, racist, sexist, toxic, dangerous or illegal content. Please make sure that your answers are socially unbiased and positive.
If a question does not make sense or is not factually coherent, please explain why, rather than answering something incorrectly.<</SYS>>
"""
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT_2 +E_SYS

In [ ]:
def get_prompt(instruction):
    return B_INST + SYSTEM_PROMPT + instruction + E_INST

os.listdir("/Users/josi/Llama2_weights")

In [ ]:
llm = LlamaCpp(
    #model_path="/Users/josi/Llama2_weights/llama-2-7b.Q4_K_M.gguf?download=true",
    model_path = "/Users/josi/Llama2_weights/llama-2-7b-chat.Q5_K_M.gguf",
    temperature=0.75,
    max_tokens=2048,
    top_p=1,
    # callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

In [ ]:
model_answers = []
def evaluate_model(model, questions):
    correct_answers = 0
    total_questions = len(questions)

    for question, expected_answer in questions.items():
        # Get the model's answer for the current question
        prompt = get_prompt(question)
        model_answer = model(prompt)

        # Remove trailing dots from the model's answer and Compare the model's answer to the expected answer (case-insensitive, whitespace-insensitive)
        model_answer = model_answer.rstrip('.')
        if model_answer.strip().lower() == expected_answer.strip().lower():
            correct_answers += 1

        # Check if any of the expected words are present in the model's answer
        #if any(word.lower() in model_answer.lower() for word in expected_answer.split()):
        #    correct_answers += 1

        # Print the results for each question
        print(f"Question: {question}")
        print(f"Expected Answer: {expected_answer}")
        print(f"Model Answer: {model_answer}")
        print("------------")

        # Store the model's answer and the corresponding question in the array
        model_answers.append({
            "question": question,
            "expected_answer": expected_answer,
            "model_answer": model_answer,
        })

    # Print overall evaluation results
    print(f"Total Questions: {total_questions}")
    print(f"Correct Answers: {correct_answers}")
    print(f"Accuracy: {correct_answers / total_questions * 100:.2f}%")


# Dictionary with questions
test_questions = {
    "What is the capital of France?": "The capital of France is Paris",
    "Translate 'hello' to Spanish.": "Hola",
    "Write an informal greeting.": "Hey there!",
    "Explain the concept of gravity.": "Gravity is a force that attracts two objects with mass",
    "What is the square root of 25?": "The square root of 25 is 5",
    "Translate 'good morning' to French.": "Bonjour",
    "Translate 'Thank you' to German.": "Danke",
    "How do you say 'hello' in Japanese?": "Konnichiwa (こんにちは)",
    "Provide the Italian translation for 'book'.": "Libro",
    "Write an informal response to the question: 'How's it going?'": "Hey! It's going well, thanks for asking! How about you?",
    "Explain the concept of artificial intelligence in an academic style.": "Artificial Intelligence (AI) refers to the development of algorithms and computational systems that enable machines to exhibit intelligent behavior, learn from experience, and perform tasks traditionally requiring human cognitive abilities. It encompasses a broad range of techniques, including machine learning, natural language processing, and expert systems. In an academic context, AI research focuses on understanding, modeling, and implementing intelligent agents capable of reasoning, problem-solving, and adapting to complex environments. The goal is to create systems that emulate human-like intelligence, allowing machines to autonomously analyze data, make informed decisions, and continuously improve their performance across diverse domains",
    "Write a casual message inviting someone to an event.": "Hey, want to join us for an event this weekend?",
}


In [ ]:
evaluate_model(llm, test_questions)

In [ ]:
print(model_answers)

In [ ]:

# Sample Test Suite
test_suite = {
    "Test1": "Question 1",
    "Test2": "Question 2",
    "Test3": "Question 3",
    "Test4": "Question 4",
    "Test5": "Question 5",
}

# Sample Predictions from the Language Model
predictions = {
    "Test1": "Answer 1",
    "Test2": "Answer 2",
    "Test3": "Answer 3",
    "Test4": "Answer 4",
    "Test5": "Answer 5",
}

# Function to Calculate the Accuracy of the Model
def accuracy(test_suite, predictions):
    correct = 0
    total = 0
    for test in test_suite:
        total += 1
        if test_suite[test] == predictions[test]:
            correct += 1
    return correct / total

# Evaluate the Model's Performance
print("Accuracy: ", accuracy(test_suite, predictions))

